In [1]:
# Dataset : http://cb.lk/covid_19

!wget http://cb.lk/covid_19


--2021-06-11 12:30:55--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 172.67.187.81, 104.21.72.203, 2606:4700:3031::6815:48cb, ...
Connecting to cb.lk (cb.lk)|172.67.187.81|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2021-06-11 12:30:55--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|172.67.187.81|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2021-06-11 12:30:56--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2021

In [2]:
!unzip covid_19

Archive:  covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-bbb6-ffe7

In [3]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"



In [4]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

In [5]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [12]:
# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [26]:

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 24,
    class_mode = 'binary')


Found 224 images belonging to 2 classes.


In [14]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [27]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 24,
    class_mode = 'binary')

Found 60 images belonging to 2 classes.


In [28]:
hist = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/10
8/8 [==============================] - 12s 2s/step - loss: 0.2517 - accuracy: 0.9091 - val_loss: 0.1923 - val_accuracy: 0.9583
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.1130 - accuracy: 0.9659 - val_loss: 0.0341 - val_accuracy: 1.0000
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.1236 - accuracy: 0.9545 - val_loss: 0.0429 - val_accuracy: 0.9792
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.1015 - accuracy: 0.9716 - val_loss: 0.0520 - val_accuracy: 0.9792
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.1205 - accuracy: 0.9489 - val_loss: 0.0569 - val_accuracy: 0.9792
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.1103 - accuracy: 0.9773 - val_loss: 0.0775 - val_accuracy: 0.9583
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.0614 - accuracy: 0.9659 - val_loss: 0.1077 - val_accuracy: 0.9583
Epoch 8/10
8/8 [========================

In [29]:

#Getting summary
summary=hist.history
print(summary)

{'loss': [0.2517443597316742, 0.11300531029701233, 0.1236141175031662, 0.10145058482885361, 0.1204724982380867, 0.11031520366668701, 0.061371609568595886, 0.10195515304803848, 0.1319524049758911, 0.04310472682118416], 'accuracy': [0.9090909361839294, 0.9659090638160706, 0.9545454382896423, 0.9715909361839294, 0.9488636255264282, 0.9772727489471436, 0.9659090638160706, 0.9659090638160706, 0.9602272510528564, 0.9886363744735718], 'val_loss': [0.19227635860443115, 0.03408663347363472, 0.04287657141685486, 0.051964472979307175, 0.056897860020399094, 0.07745378464460373, 0.10774987936019897, 0.037918638437986374, 0.05020090937614441, 0.05214298889040947], 'val_accuracy': [0.9583333134651184, 1.0, 0.9791666865348816, 0.9791666865348816, 0.9791666865348816, 0.9583333134651184, 0.9583333134651184, 1.0, 0.9583333134651184, 0.9791666865348816]}


In [30]:
model.save("model_covid.h5")


In [31]:
model.evaluate_generator(train_generator)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.04988064616918564, 0.9776785969734192]

In [32]:
print(model.evaluate_generator(validation_generator))


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.041993144899606705, 0.9833333492279053]
